# Multinomial Predictions From Binary Model Estimations

In [1]:
import os
import random
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt
%matplotlib inline 

# remove warnings
import warnings
warnings.filterwarnings('ignore')


C:\Users\peter\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Binary Logistic Regression (BNL) can be used to estimate models with a binary target. Multinomial Logistic Regression (MNL) can be used to estimate models with discrete targets having 2 or more unordered pairs. While the BNL method is a special case of MNL, it is possible to specify and estimate a MNL model using separate BNL models for $J-1$ categories where $J$ is the reference category.

This method is usually credited to Begg & Gray (1984) and its use is explained in detail in popular statistics texts (Allison, 2012; Hosemer & Lemeshow, 2004; Agresti, 2011).

In the credit risk literature, it has been used to estimate delinquency states in transition matrices for mortgage portfolios (Grimshaw, et al. 2014; Constantinou, et al. 2010), model bank M&A outcomes (Koetter, et al. 2007), and is presented as a general method for estimating default and prepayment options (Castelli, 2012).

This may not be an ideal option as there is a loss of efficiency, however, in certain circumstances it is preferable for computational reasons or to avoid convergence issues.

### Estimation
To use this approach, estimate $J-1$ binary logit models where for each model we reduce the training dataset to observations having the reference event type and the event type of interest only. In this way we are estimating models that compare
$ P(Y=j|x) $ to $P(Y=J|x)$. See Agresti (2011) for more information on Baseline-Category Logits.

### Model Correction
While Begg & Gray (1984) showed that it is possible to estimate effects in this way, to make predictions, we need to correct the raw BNL model outputs. To understand this, note that the MNL model can be described in terms of the softmax function:

$$ P(Y=j|X) = \frac{e^{X\beta_j}}{\sum_{k=1}^{J} e^{X\beta_k} }$$

Note that with the Begg & Gray approach, we estimate models comparing the $j$th category to a baseline $J$ category. Since $e^{X\beta_j} = \frac{P(Y=j|X)}{P(Y=J|X)}$, is exactly what we estimate with the binary models, we simply take the sum of the separate $e^{X\beta_j}$ estimated by binary models as the deminator to recover the MNL (softmax) probabilities.

The example below describes this proces

## Simulate data

In [2]:
def sim_mnl(n, npreds, nlevels_of_y, rseed=212):
    '''
    simulates data based on a multinomial logistic regression (softmax) model
    returns data (df) and the true coefficients (beta)
    :param n: number of observations
    :param npreds: number of predictors in x
    :param nlevels_of_y: number of levels in y (j=1...J)
    :param rseed: random seed
    '''
    random.seed(rseed)
    np.random.seed(rseed)
    # simulate standard normal covariates + regression coefficients
    x = np.random.normal(size=[n, npreds])
    beta = np.random.normal(size=[npreds, (nlevels_of_y-1)])
    # n rows, nlevels_of_y-1 cols
    e_xbeta = np.exp(x.dot(beta))
    # $1 / (1 + \sum_{j=1}^{nlevels_of_y-1} \exp{X_j\upbeta})$
    inv_softmax_denom = 1 / (np.sum(e_xbeta, axis=1) + 1)
    inv_softmax_denom = inv_softmax_denom[:, np.newaxis] # adds a new axis -> 2D array
    # true probs for nlevels_of_y-1 models
    p_true = np.multiply(e_xbeta, inv_softmax_denom)
    # add complement so rows sum to 1
    z = 1 - np.sum(p_true, axis=1)
    z = z[:, np.newaxis]
    p_true = np.append(p_true, z, axis=1)
    # y - outcome
    y = list()
    for i in range(n):
        outcome = np.random.multinomial(n=1,pvals=p_true[i,:]).argmax() + 1
        y.append(outcome)
    # return pandas dataframe of simulated data
    df = pd.DataFrame(x)
    df.columns = 'x' + df.columns.astype(str)  # rename cols x1, x2, ...
    df['y'] = np.where(np.array(y)==3, 0, y)   # set last level (nlevels_of_y) to be 0
    return(df, beta)

In [3]:
# simulate data
df, beta = sim_mnl(n=1000, npreds=5, nlevels_of_y=3, rseed=212)

# break into training and testing data
# msk = np.random.rand(len(df)) < 0.5
# df_train = df[msk]
# df_test = df[~msk]

# automatically create model formula
mod_formula = 'y~' + '+'.join([i for i in df.columns if i!='y'])

# remove to avoid accidentally using
# del(df)

## Fit MNL Model

In [4]:
def fit_multinomial(data):
    # generate endogenous and exogenous 
    Y, X = dmatrices(mod_formula, data=data, return_type='dataframe')
    # fit model    
    model = sm.MNLogit(Y, X)
    results = model.fit()
    return model, results

mod_mnl, rslt_mnl = fit_multinomial(data=df)

Optimization terminated successfully.
         Current function value: 0.848411
         Iterations 6


### MNL Model Summary

In [5]:
rslt_mnl.summary()

AttributeError: module 'scipy.stats' has no attribute 'chisqprob'

### MNL Predicted Values

In [ ]:
Y, X = dmatrices(mod_formula, data=df, return_type='dataframe')
beta = rslt_mnl.params

# calculate manually
# verify equal to rslt_mnl.predict(X)
e_xbeta = np.exp(np.dot(X,beta))
inv_softmax_denom = 1 / (np.sum(e_xbeta, axis=1) + 1)
inv_softmax_denom = inv_softmax_denom[:, np.newaxis] # adds a new axis -> 2D array
pred_mnl = np.multiply(e_xbeta, inv_softmax_denom)

### Verify the sum of predicted values adds up to the count of each category of Y

In [ ]:
print(sum(pred_mnl))
df['y'].value_counts()

## Fit Separate BNL Models

In [ ]:
bool1 = df['y'].isin([0,1])
bool2 = df['y'].isin([0,2])

def fit_binomial(data):
    # generate endogenous and exogenous 
    Y, X = dmatrices(mod_formula, data=data, return_type='dataframe')
    # set highest value of Y to be 1
    Y = np.where(Y > 0, 1, 0)
    # fit model
    model = sm.Logit(Y, X)
    results = model.fit()
    return model, results
    
mod_bnl1, rslt_bnl1 = fit_binomial(data=df[bool1])
mod_bnl2, rslt_bnl2 = fit_binomial(data=df[bool2])

### Compare MNL and BNL Coefficients
Note coefficients for all inputs are very close, however, the intercepts are not necessarily close.

In [ ]:
def compare_fits(rslt_mnl, rslt_bnl1, rslt_bnl2):
    z = rslt_mnl.params
    z.columns = ['mnl1', 'mnl2']
    y = rslt_bnl1.params
    y.name = 'bnl1'
    z = z.join(y)
    y = rslt_bnl2.params
    y.name = 'bnl2' 
    z = z.join(y)
    return(z)

compare_fits(rslt_mnl, rslt_bnl1, rslt_bnl2)

### Using the software to predict MNL probs from BNL estimated models overestimates event probabilities

In [ ]:
beta1 = rslt_bnl1.params
beta2 = rslt_bnl2.params
e_xbeta1 = np.exp(np.dot(X, beta1))
e_xbeta2 = np.exp(np.dot(X, beta2))

In [ ]:
p_bnl1_raw = e_xbeta1 / (1 + e_xbeta1)
# Equivalent to: rslt_bnl1.predict(X).sum()
p_bnl1_raw.sum()

### Instead, need to use softmax to correct probabilities

Need to add exp(logit) from all other (non reference) categories of Y. Note the sums of the predicted probabilities are close but not exactly equal to their multinomial equivalents and the true event counts. There is some loss of efficiency due to estimating models on smaller samples in the binary models.

In [ ]:
p_bnl1_adj = e_xbeta1 / (1 + e_xbeta1 + e_xbeta2)
p_bnl1_adj.sum()

In [ ]:
p_bnl2_adj = e_xbeta2 / (1 + e_xbeta1 + e_xbeta2)
p_bnl2_adj.sum()

In [ ]:
df['y'].value_counts()